In [7]:
import pandas as pd
import requests
import datetime as dt
from dateutil import parser
import pytz
import json

import os
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [8]:

API_ID = os.environ.get("API_ID")
API_KEY = os.environ.get("API_KEY")
API_SECRET = os.environ.get("API_SECRET")
OPENFX_URL = os.environ.get("OPENFX_URL")

SECURE_HEADER = {
    "Authorization": f"Basic {API_ID}:{API_KEY}:{API_SECRET}",
    "Content-Type": "application/json",
    "Accept": "application/json",
}

In [9]:
session = requests.Session()
session.headers.update(SECURE_HEADER)

In [10]:
full_url = lambda x: f"{OPENFX_URL}/{x}"

In [11]:
full_url('account')

'https://marginalttdemowebapi.fxopen.net/api/v2/account'

## Account Details

In [50]:
resp = session.get(full_url('account'))

In [51]:
resp.status_code

200

In [52]:
print(json.dumps(resp.json(), indent=2))

{
  "Id": 28501564,
  "AccountingType": "Gross",
  "Name": "mauqbK8EqB",
  "FirstName": "",
  "LastName": "",
  "Phone": "+5586981550215",
  "Country": "BRAZIL",
  "State": "",
  "City": "",
  "Address": "",
  "ZipCode": "",
  "Email": "jonatas.iw@gmail.com",
  "Comment": "",
  "Registered": 1725839019376,
  "Modified": 1725839412695,
  "IsArchived": false,
  "IsBlocked": false,
  "IsReadonly": false,
  "IsValid": true,
  "IsWebApiEnabled": true,
  "Leverage": 500,
  "Balance": 10000.0,
  "BalanceCurrency": "USD",
  "Profit": 0.0,
  "Commission": 0.0,
  "AgentCommission": 0.0,
  "Swap": 0.0,
  "Rebate": 0.0,
  "Equity": 10000.0,
  "Margin": 0.0,
  "MarginCallLevel": 100,
  "StopOutLevel": 50,
  "ReportCurrency": "USD",
  "IsTokenCommissionEnabled": false,
  "Throttling": [
    {
      "Protocol": "Rest",
      "SessionsPerAccount": 2,
      "RequestsPerSecond": 5,
      "ThrottlingMethods": [],
      "ConcurrentRequestCount": 100
    }
  ],
  "IsLongOnly": false
}


## Symbols (Instruments)

In [53]:
# first get all of the symbols
resp = session.get(full_url('symbol'))
symbol_data = resp.json()
symbol_data

[{'Symbol': 'EURUSD',
  'Precision': 5,
  'IsTradeAllowed': True,
  'MarginMode': 'Forex',
  'ProfitMode': 'Forex',
  'ContractSize': 100000.0,
  'MarginHedged': 0.5,
  'MarginFactor': 1.0,
  'MarginCurrency': 'EUR',
  'MarginCurrencyPrecision': 2,
  'ProfitCurrency': 'USD',
  'ProfitCurrencyPrecision': 2,
  'Description': 'Euro vs US Dollar',
  'Schedule': 'Forex',
  'Color': -10496,
  'SwapEnabled': True,
  'SwapType': 'Points',
  'SwapSizeShort': 0.84,
  'SwapSizeLong': -11.09,
  'TripleSwapDay': 3,
  'MinTradeAmount': 1000.0,
  'MaxTradeAmount': 100000000.0,
  'TradeAmountStep': 1000.0,
  'CommissionType': 'Percentage',
  'CommissionChargeType': 'PerLot',
  'Commission': 0.0018,
  'LimitsCommission': 0.0015,
  'MinCommission': 0.0,
  'MinCommissionCurrency': 'USD',
  'DefaultSlippage': 0.01,
  'StatusGroupId': 'Forex',
  'SecurityName': 'ECN FX Group1',
  'SecurityDescription': 'Major Forex symbols',
  'StopOrderMarginReduction': 1.0,
  'HiddenLimitOrderMarginReduction': 1.0,
  'Is

In [54]:

# print first 2, note the StatusGroupId=="Forex"
[print(json.dumps(x, indent=2)) for x in symbol_data[:2]]


{
  "Symbol": "EURUSD",
  "Precision": 5,
  "IsTradeAllowed": true,
  "MarginMode": "Forex",
  "ProfitMode": "Forex",
  "ContractSize": 100000.0,
  "MarginHedged": 0.5,
  "MarginFactor": 1.0,
  "MarginCurrency": "EUR",
  "MarginCurrencyPrecision": 2,
  "ProfitCurrency": "USD",
  "ProfitCurrencyPrecision": 2,
  "Description": "Euro vs US Dollar",
  "Schedule": "Forex",
  "Color": -10496,
  "SwapEnabled": true,
  "SwapType": "Points",
  "SwapSizeShort": 0.84,
  "SwapSizeLong": -11.09,
  "TripleSwapDay": 3,
  "MinTradeAmount": 1000.0,
  "MaxTradeAmount": 100000000.0,
  "TradeAmountStep": 1000.0,
  "CommissionType": "Percentage",
  "CommissionChargeType": "PerLot",
  "Commission": 0.0018,
  "LimitsCommission": 0.0015,
  "MinCommission": 0.0,
  "MinCommissionCurrency": "USD",
  "DefaultSlippage": 0.01,
  "StatusGroupId": "Forex",
  "SecurityName": "ECN FX Group1",
  "SecurityDescription": "Major Forex symbols",
  "StopOrderMarginReduction": 1.0,
  "HiddenLimitOrderMarginReduction": 1.0,
  "

[None, None]

In [55]:

# also we only want symbols where we can also load history data. For that there is the quotehistory/symbols
resp = session.get(full_url('quotehistory/symbols'))
his_symbol_data = resp.json()

print(len(his_symbol_data))
print(his_symbol_data[:5])

# you can probaby see, some of the instruments are appended with "_L"
# in the API code we will filter for symbols that are in the symbol_data and are in his_symbol_data and do not have this L and have StatusGroupId=="Forex"

1300
['#AUS200', '#AUS200_L', '#ESX50', '#ESX50_L', '#FCHI']


## Perdiodicities (Granularities)

In [56]:
# for a given instrument (symbol) we can get the available candle granularities
resp = session.get(full_url('quotehistory/BTCUSD/periodicities'))
print(resp.json())

['D1', 'H1', 'H4', 'M1', 'M15', 'M30', 'M5', 'MN1', 'S1', 'S10', 'W1']


## Candles

Candles are a little bit different than Oanda. <br><br>
The Good:
- Prices are floats!!
- Last available candle is in the response<br><br>

The bad:
- We have to make separate calls for ask and bid prices
- We have to specify a from date no matter what. It has to be a timestamp in ms format without timezone.
    - If the count we specify is negative it counts back from the date
    - If the count we specify is postive it counts forward from the date
- There is a 1000 candle limit per request

In [57]:
test_date = dt.datetime.now(dt.UTC)
past_date = parser.parse("2023-03-02T03:11:00")
print("test_date", test_date)
print("past_date", past_date)

test_date_ts = pd.Timestamp(test_date).timestamp()
past_date_ts = pd.Timestamp(past_date).timestamp()
print("test_date_ts", test_date_ts)
print("int(test_date_ts*1000)", int(test_date_ts*1000))
print("int(past_date_ts*1000)", int(past_date_ts*1000))

test_date 2024-09-09 18:16:39.060555+00:00
past_date 2023-03-02 03:11:00
test_date_ts 1725905799.060555
int(test_date_ts*1000) 1725905799060
int(past_date_ts*1000) 1677726660000


In [58]:
ts_conv = int(test_date_ts*1000)
pd.to_datetime(ts_conv, unit='ms')

Timestamp('2024-09-09 18:16:39.060000')

In [59]:
LABEL_MAP = {
    'Open': 'o',
    'High': 'h',
    'Low': 'l',
    'Close': 'c',
}

# normal params
count = -10
granularity = "M15"
pair = "EURUSD"

# how far do we need to go back to get our candles
params = dict(
    timestamp=int(pd.Timestamp(dt.datetime.now(dt.UTC)).timestamp() * 1000),
    count=count
)

url = full_url(f'quotehistory/{pair}/{granularity}/bars/bid')
bid_data = session.get(url, params=params).json()

url = full_url(f'quotehistory/{pair}/{granularity}/bars/ask')
ask_data = session.get(url, params=params).json()


In [60]:
bid_data

{'From': 1725897600000,
 'To': 1725905700000,
 'Bars': [{'Timestamp': 1725897600000,
   'Open': 1.10414,
   'High': 1.10461,
   'Low': 1.10414,
   'Close': 1.10449,
   'Volume': 1473.0},
  {'Timestamp': 1725898500000,
   'Open': 1.10448,
   'High': 1.10487,
   'Low': 1.10445,
   'Close': 1.10481,
   'Volume': 1349.0},
  {'Timestamp': 1725899400000,
   'Open': 1.1048,
   'High': 1.10499,
   'Low': 1.1047,
   'Close': 1.10486,
   'Volume': 1360.0},
  {'Timestamp': 1725900300000,
   'Open': 1.10487,
   'High': 1.10507,
   'Low': 1.10474,
   'Close': 1.10487,
   'Volume': 1416.0},
  {'Timestamp': 1725901200000,
   'Open': 1.10487,
   'High': 1.10531,
   'Low': 1.10481,
   'Close': 1.10529,
   'Volume': 1206.0},
  {'Timestamp': 1725902100000,
   'Open': 1.10527,
   'High': 1.10531,
   'Low': 1.10512,
   'Close': 1.10517,
   'Volume': 1237.0},
  {'Timestamp': 1725903000000,
   'Open': 1.10516,
   'High': 1.10529,
   'Low': 1.10489,
   'Close': 1.10489,
   'Volume': 1188.0},
  {'Timestamp': 1

In [61]:
# bid_data and ask_data will contain (hopefully) a key called "Bars", with the candle data:
bid_data["Bars"][:2] # the first two

[{'Timestamp': 1725897600000,
  'Open': 1.10414,
  'High': 1.10461,
  'Low': 1.10414,
  'Close': 1.10449,
  'Volume': 1473.0},
 {'Timestamp': 1725898500000,
  'Open': 1.10448,
  'High': 1.10487,
  'Low': 1.10445,
  'Close': 1.10481,
  'Volume': 1349.0}]

In [62]:
# Now to convert them to a dataframe
# main points: Timestamp to a datetime.
# we'll need to make a DataFrame for ask, for bid, merge and calculate the mid

In [63]:
# a little utility to take in a candle and return it as and object
# for example, if we are working with bid prices
# price_label='bid'
# item= {'Volume': 1476, 'Close': 1.06064,  'Low': 1.06054,  'High': 1.06104,  'Open': 1.06081,  'Timestamp': 1677535200000}
# the returned object is: { 'time': datetime, 'bid_c': 1.06064,  'bid_l': 1.06054,  'bid_h': 1.06104,  'bid_o': 1.06081 }
def get_price_dict(price_label: str, item):
        data = dict(time=pd.to_datetime(item['Timestamp'], unit='ms'))
        for ohlc in LABEL_MAP.keys():
            data[f"{price_label}_{LABEL_MAP[ohlc]}"]=item[ohlc]
        return data

In [64]:
# let's make the lists of objects
AvailableTo = pd.to_datetime(bid_data['AvailableTo'], unit='ms')

bids = [get_price_dict('bid', item) for item in bid_data["Bars"]]
asks = [get_price_dict('ask', item) for item in ask_data["Bars"]]

In [65]:
# last 2
bids[-2:]

[{'time': Timestamp('2024-09-09 18:00:00'),
  'bid_o': 1.10436,
  'bid_h': 1.10446,
  'bid_l': 1.10414,
  'bid_c': 1.10425},
 {'time': Timestamp('2024-09-09 18:15:00'),
  'bid_o': 1.10426,
  'bid_h': 1.10432,
  'bid_l': 1.10424,
  'bid_c': 1.10432}]

In [66]:
# now merge on time - the assumption here is we have the same time values for both. it would be weird if we didn't
df_bid = pd.DataFrame.from_dict(bids)
df_ask = pd.DataFrame.from_dict(asks)
df_merged = pd.merge(left=df_bid, right=df_ask, on='time')    

In [67]:
df_merged

,time,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-09-09 16:00:00,1.10414,1.10461,1.10414,1.10449,1.10416,1.10460,1.10416,1.10450
1,2024-09-09 16:15:00,1.10448,1.10487,1.10445,1.10481,1.10448,1.10489,1.10447,1.10483
2,2024-09-09 16:30:00,1.10480,1.10499,1.10470,1.10486,1.10482,1.10501,1.10471,1.10488
3,2024-09-09 16:45:00,1.10487,1.10507,1.10474,1.10487,1.10488,1.10509,1.10475,1.10488
4,2024-09-09 17:00:00,1.10487,1.10531,1.10481,1.10529,1.10489,1.10527,1.10482,1.10527
5,2024-09-09 17:15:00,1.10527,1.10531,1.10512,1.10517,1.10527,1.10527,1.10514,1.10519
6,2024-09-09 17:30:00,1.10516,1.10529,1.10489,1.10489,1.10518,1.10527,1.10485,1.10488
7,2024-09-09 17:45:00,1.10489,1.10493,1.10434,1.10435,1.10489,1.10494,1.10435,1.10436
8,2024-09-09 18:00:00,1.10436,1.10446,1.10414,1.10425,1.10437,1.10448,1.10416,1.10427
9,2024-09-09 18:15:00,1.10426,1.10432,1.10424,1.10432,1.10427,1.10434,1.10426,1.10433


In [68]:
# FINALLY calcuate the mid, and we are done
for i in ['_o', '_h', '_l', '_c']:
    df_merged[f'mid{i}'] = (df_merged[f'ask{i}'] - df_merged[f'bid{i}']) / 2 + df_merged[f'bid{i}']

In [69]:
df_merged

,time,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,mid_o,mid_h,mid_l,mid_c
0,2024-09-09 16:00:00,1.10414,1.10461,1.10414,1.10449,1.10416,1.10460,1.10416,1.10450,1.104150,1.104605,1.104150,1.104495
1,2024-09-09 16:15:00,1.10448,1.10487,1.10445,1.10481,1.10448,1.10489,1.10447,1.10483,1.104480,1.104880,1.104460,1.104820
2,2024-09-09 16:30:00,1.10480,1.10499,1.10470,1.10486,1.10482,1.10501,1.10471,1.10488,1.104810,1.105000,1.104705,1.104870
3,2024-09-09 16:45:00,1.10487,1.10507,1.10474,1.10487,1.10488,1.10509,1.10475,1.10488,1.104875,1.105080,1.104745,1.104875
4,2024-09-09 17:00:00,1.10487,1.10531,1.10481,1.10529,1.10489,1.10527,1.10482,1.10527,1.104880,1.105290,1.104815,1.105280
5,2024-09-09 17:15:00,1.10527,1.10531,1.10512,1.10517,1.10527,1.10527,1.10514,1.10519,1.105270,1.105290,1.105130,1.105180
6,2024-09-09 17:30:00,1.10516,1.10529,1.10489,1.10489,1.10518,1.10527,1.10485,1.10488,1.105170,1.105280,1.104870,1.104885
7,2024-09-09 17:45:00,1.10489,1.10493,1.10434,1.10435,1.10489,1.10494,1.10435,1.10436,1.104890,1.104935,1.104345,1.104355
8,2024-09-09 18:00:00,1.10436,1.10446,1.10414,1.10425,1.10437,1.10448,1.10416,1.10427,1.104365,1.104470,1.104150,1.104260
9,2024-09-09 18:15:00,1.10426,1.10432,1.10424,1.10432,1.10427,1.10434,1.10426,1.10433,1.104265,1.104330,1.104250,1.104325


In [70]:
if count < 0 and df_merged.shape[0] > 0 and df_merged.iloc[-1].time == AvailableTo:
    df_merged = df_merged[:-1]

In [71]:
df_merged

,time,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,mid_o,mid_h,mid_l,mid_c
0,2024-09-09 16:00:00,1.10414,1.10461,1.10414,1.10449,1.10416,1.10460,1.10416,1.10450,1.104150,1.104605,1.104150,1.104495
1,2024-09-09 16:15:00,1.10448,1.10487,1.10445,1.10481,1.10448,1.10489,1.10447,1.10483,1.104480,1.104880,1.104460,1.104820
2,2024-09-09 16:30:00,1.10480,1.10499,1.10470,1.10486,1.10482,1.10501,1.10471,1.10488,1.104810,1.105000,1.104705,1.104870
3,2024-09-09 16:45:00,1.10487,1.10507,1.10474,1.10487,1.10488,1.10509,1.10475,1.10488,1.104875,1.105080,1.104745,1.104875
4,2024-09-09 17:00:00,1.10487,1.10531,1.10481,1.10529,1.10489,1.10527,1.10482,1.10527,1.104880,1.105290,1.104815,1.105280
5,2024-09-09 17:15:00,1.10527,1.10531,1.10512,1.10517,1.10527,1.10527,1.10514,1.10519,1.105270,1.105290,1.105130,1.105180
6,2024-09-09 17:30:00,1.10516,1.10529,1.10489,1.10489,1.10518,1.10527,1.10485,1.10488,1.105170,1.105280,1.104870,1.104885
7,2024-09-09 17:45:00,1.10489,1.10493,1.10434,1.10435,1.10489,1.10494,1.10435,1.10436,1.104890,1.104935,1.104345,1.104355
8,2024-09-09 18:00:00,1.10436,1.10446,1.10414,1.10425,1.10437,1.10448,1.10416,1.10427,1.104365,1.104470,1.104150,1.104260


In [72]:
# and breathe...

## Latest Prices

In [73]:
# here the endpoint needs space separated instruments rather than comma. Yes, in a URL that is a bit weird.
instruments_list = ["GBPJPY", "EURUSD", "EURNOK"]
url = full_url(f"tick/{' '.join(instruments_list)}")
print(url)

https://marginalttdemowebapi.fxopen.net/api/v2/tick/GBPJPY EURUSD EURNOK


In [74]:
prices = session.get(url)
price_data = prices.json()

# you can see below that there are some differences to the Oanda Api in what comes back, imho this is a lot better
price_data

[{'Symbol': 'EURUSD',
  'Timestamp': 1725905805128,
  'BestBid': {'Type': 'Bid', 'Price': 1.10432, 'Volume': 125000000.0},
  'BestAsk': {'Type': 'Ask', 'Price': 1.10433, 'Volume': 125000000.0},
  'IndicativeTick': False,
  'TickType': 'Normal'},
 {'Symbol': 'GBPJPY',
  'Timestamp': 1725905805229,
  'BestBid': {'Type': 'Bid', 'Price': 186.874, 'Volume': 250000000.0},
  'BestAsk': {'Type': 'Ask', 'Price': 186.886, 'Volume': 250000000.0},
  'IndicativeTick': False,
  'TickType': 'Normal'},
 {'Symbol': 'EURNOK',
  'Timestamp': 1725905804825,
  'BestBid': {'Type': 'Bid', 'Price': 11.95899, 'Volume': 50000000.0},
  'BestAsk': {'Type': 'Ask', 'Price': 11.96272, 'Volume': 6500000.0},
  'IndicativeTick': False,
  'TickType': 'Normal'}]

In [79]:
pd.to_datetime("1725905805128", unit='ms')

/tmp/ipykernel_31579/3042125448.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  pd.to_datetime("1725905805128", unit='ms')


Timestamp('2024-09-09 18:16:45.128000')

In [81]:
with open("price_data.json", "w") as f:
    f.write(json.dumps(price_data, indent=2))